### Import libraries

In [20]:
# Import Libraries
import numpy as np
import csv

### Customize

In [21]:
# Provide parameters

my_discount_rate = 0.10
my_hurdle_rate = 0.15
my_new_project_cf = [-10000000, 3000000, 3500_000, 4000000, 4500000, 5000000]
my_upgrade_project_cf = [-12000000, 3400000, 4000000, 4600000, 5200000, 5800000]

csv_file= '0708 project_evaluation_results.csv'
text_file= '0708 project_evaluation_results.txt'

### Define functions

In [22]:
# Compute NPV
def compute_npv(cash_flows, discount_rate):
    npv = sum(cf / ((1 + discount_rate) ** i) for i, cf in enumerate(cash_flows))
    return npv

# Compute IRR
def compute_irr(cash_flows, guess=0.1, max_iterations=100, tolerance=1e-6):
    def npv_derivative(cash_flows, r):
        return -sum(i * cf / ((1 + r) ** (i + 1)) for i, cf in enumerate(cash_flows) if i > 0)

    r = guess
    for _ in range(max_iterations):
        f_value = sum(cf / ((1 + r) ** i) for i, cf in enumerate(cash_flows))
        f_deriv = npv_derivative(cash_flows, r)
        if abs(f_deriv) < 1e-12:
            break
        new_r = r - f_value / f_deriv
        if abs(new_r - r) < tolerance:
            return new_r
        r = new_r
    return r

# Compute discounted payback period
def compute_discounted_payback_period(cash_flows, discount_rate):
    cumulative = 0.0
    for i, cf in enumerate(cash_flows):
        discounted_cf = cf / ((1 + discount_rate) ** i)
        cumulative += discounted_cf
        if cumulative >= 0:
            previous_cumulative = cumulative - discounted_cf
            fraction = (0 - previous_cumulative) / discounted_cf if discounted_cf != 0 else 0
            return i - 1 + fraction if i != 0 else 0.0
    return None

# Saving results to txt file
def save_results_to_file(filename, discount_rate, hurdle_rate,
                         new_project_cf, upgrade_project_cf,
                         incremental_cf, npv_new, irr_new, payback_new,
                         npv_upgrade, irr_upgrade, payback_upgrade,
                         npv_incr, irr_incr, payback_incr,
                         max_initial_investment, reduction_needed,
                         min_probability):
    with open(filename, 'w') as f:
        f.write("Financial Evaluation Results\n")
        f.write("=" * 40 + "\n\n")
        f.write(f"Discount Rate: {discount_rate*100:.2f}%\n")
        f.write(f"Hurdle Rate (IRR Benchmark): {hurdle_rate*100:.2f}%\n\n")

        for label, npv, irr, payback in [("New Project", npv_new, irr_new, payback_new),
                                         ("Upgrade Project", npv_upgrade, irr_upgrade, payback_upgrade),
                                         ("Incremental Project", npv_incr, irr_incr, payback_incr)]:
            f.write(f"{label}:\n")
            f.write(f"  NPV: ${npv:,.2f}\n")
            f.write(f"  IRR: {irr*100:.2f}%\n")
            f.write(f"  Discounted Payback Period: {payback:.2f} years\n" if payback else "  Never breaks even\n")
            f.write("\n")

        if irr_incr < hurdle_rate:
            f.write(f"Incremental IRR ({irr_incr*100:.2f}%) BELOW hurdle rate ({hurdle_rate*100:.2f}%).\n")
            f.write(f"Max initial investment to meet hurdle: ${abs(max_initial_investment):,.2f}\n")
            f.write(f"Current initial investment: ${abs(incremental_cf[0]):,.2f}\n")
            f.write(f"Reduction needed: ${reduction_needed:,.2f}\n")
        else:
            f.write(f"Incremental IRR ({irr_incr*100:.2f}%) ABOVE hurdle rate ({hurdle_rate*100:.2f}%).\n")

        f.write("\n")
#        f.write(f"Min probability for positive NPV: {min_probability:.4f} ({min_probability*100:.2f}%)\n" if min_probability else "No meaningful probability threshold.\n")

        if min_probability is not None and 0 <= min_probability <= 1:
            f.write(f"Min probability for positive NPV: {min_probability:.4f} ({min_probability*100:.2f}%)\n")
        else:
            f.write("No meaningful probability threshold.\n")


def save_results_to_csv(filename, results):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Project', 'NPV', 'IRR (%)', 'Discounted Payback Period (years)'])
        for row in results:
            writer.writerow(row)

# Define the main computational engine performing the main tasks with values provided 

def main():
    discount_rate = my_discount_rate
    hurdle_rate = my_hurdle_rate
    new_project_cf = my_new_project_cf
    upgrade_project_cf = my_upgrade_project_cf

    incremental_cf = [u - n for u, n in zip(upgrade_project_cf, new_project_cf)]

    # computing investment metrices for standard project
    npv_new = compute_npv(new_project_cf, discount_rate)
    irr_new = compute_irr(new_project_cf)
    payback_new = compute_discounted_payback_period(new_project_cf, discount_rate)

    # computing investment metrices for upgraded project
    npv_upgrade = compute_npv(upgrade_project_cf, discount_rate)
    irr_upgrade = compute_irr(upgrade_project_cf)
    payback_upgrade = compute_discounted_payback_period(upgrade_project_cf, discount_rate)

    # computing investment metrices for incremental project
    npv_incr = compute_npv(incremental_cf, discount_rate)
    irr_incr = compute_irr(incremental_cf)
    payback_incr = compute_discounted_payback_period(incremental_cf, discount_rate)

    # Computing alterations required if incrementa project is not efficient 
    max_initial_investment = reduction_needed = min_probability = None
    if irr_incr < hurdle_rate:
        sum_future_pv = sum(incremental_cf[i] / ((1 + hurdle_rate) ** i) for i in range(1, len(incremental_cf)))
        max_initial_investment = -sum_future_pv
        reduction_needed = abs(incremental_cf[0]) - abs(max_initial_investment)

    pv_future = sum(incremental_cf[i] / ((1 + discount_rate) ** i) for i in range(1, len(incremental_cf)))
    if pv_future > 0:
        min_probability = -incremental_cf[0] / pv_future

    # save results to text file
    save_results_to_file(text_file, discount_rate, hurdle_rate,
                         new_project_cf, upgrade_project_cf, incremental_cf,
                         npv_new, irr_new, payback_new,
                         npv_upgrade, irr_upgrade, payback_upgrade,
                         npv_incr, irr_incr, payback_incr,
                         max_initial_investment, reduction_needed, min_probability)

    # save results to csv file
    save_results_to_csv(csv_file, [
        ['New Project', npv_new, irr_new*100, payback_new],
        ['Upgrade Project', npv_upgrade, irr_upgrade*100, payback_upgrade],
        ['Incremental Project', npv_incr, irr_incr*100, payback_incr]
    ])

    # Display results on screen
    with open('0708 project_evaluation_results.txt', 'r') as f:
        print(f.read())


### Run the main module

In [23]:
# Validation
if not any(cf < 0 for cf in my_new_project_cf):
    raise SystemExit("Cash outflow not defined in new project. Please provide the same and retry.")

if not any(cf < 0 for cf in my_upgrade_project_cf):
    raise SystemExit("Cash outflow not defined in new project. Please provide the same and retry.")

# Your main logic continues here...
print("✅ Cash outflows found. Proceeding with evaluation...")

# running the main module
if __name__ == "__main__":
    main()

✅ Cash outflows found. Proceeding with evaluation...
Financial Evaluation Results

Discount Rate: 10.00%
Hurdle Rate (IRR Benchmark): 15.00%

New Project:
  NPV: $4,803,261.08
  IRR: 25.75%
  Discounted Payback Period: 3.45 years

Upgrade Project:
  NPV: $5,005,755.94
  IRR: 23.76%
  Discounted Payback Period: 3.60 years

Incremental Project:
  NPV: $202,494.86
  IRR: 13.45%
  Discounted Payback Period: 4.59 years

Incremental IRR (13.45%) BELOW hurdle rate (15.00%).
Max initial investment to meet hurdle: $1,918,376.32
Current initial investment: $2,000,000.00
Reduction needed: $81,623.68

Min probability for positive NPV: 0.9081 (90.81%)

